<a href="https://colab.research.google.com/github/CharlesPoletowin/YCBS-273/blob/master/RNN_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
from sklearn.metrics import accuracy_score

This notebook was inspired from https://github.com/bentrevett/pytorch-sentiment-analysis. Great thanks to the authors!

# Data setup

In [0]:
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

def get_data(seq_len, inp_dim, device, data_size=25000):

  data = torch.randint(low=0, high=inp_dim, size=(data_size, seq_len), out=None, device=device)
  labels = torch.abs(data[:, 0])

  train_data = TensorDataset(data[:int(0.7*data_size)], labels[:int(0.7*data_size)])
  valid_data = TensorDataset(data[int(0.7*data_size): int(0.85*data_size)], labels[int(0.7*data_size): int(0.85*data_size)])
  test_data = TensorDataset(data[int(0.85*data_size): int(data_size)],  labels[int(0.85*data_size): int(data_size)])

  train_data_loader = DataLoader(train_data, batch_size=64)
  valid_data_loader = DataLoader(valid_data, batch_size=64)
  test_data_loader = DataLoader(test_data, batch_size=64)
  
  return train_data_loader, valid_data_loader, test_data_loader

# Model definition

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, inp):

        #inp = [seq len, batch size]
        
        embedded = self.embedding(inp)
        
        #embedded = [seq len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [seq len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        print(hidden.shape)
        
        return self.fc(hidden[0])

class RNN_deep(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers=2):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, num_layers=n_layers)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, inp):

        #inp = [seq len, batch size]
        
        embedded = self.embedding(inp)
        
        #embedded = [seq len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [seq len, batch size, hid dim]
        #hidden = [num_layers, batch size, hid dim]
        
        print(hidden.shape)
        hidden=hidden.view(hidden.shape[1],hidden.shape[0]*hidden.shape[2])
        print(hidden.shape)
        
        return self.fc(hidden[-1])

class RNN_deep_bidir(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, n_layers=1):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, 
                           num_layers=n_layers,
                           bidirectional=True)
        
        self.fc = nn.Linear(2 * hidden_dim, output_dim)
        
    def forward(self, text):#, text_lengths):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
                
        #output = [sent len, batch size, hid dim * num directions]
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        print(hidden.shape)

        hidden=torch.cat((hidden[-2,:,:],hidden[-1,:,:]),dim=1)

        print(hidden.shape)
        
        return self.fc(hidden)

class LSTM(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, inp):

        #inp = [seq len, batch size]
        
        embedded = self.embedding(inp)
        
        #embedded = [seq len, batch size, emb dim]
        
        output, (hidden, cell_state) = self.lstm(embedded)
        
        #output = [seq len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        return self.fc(hidden.squeeze(0))

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [0]:
def evaluate(model, data_iterator, loss_func):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for inp, label in data_iterator:

            predictions = model(inp.t()).squeeze(1)
            
            loss = loss_func(predictions, label)
            
            acc = accuracy_score(torch.argmax(predictions, dim=1).cpu().detach().numpy(), label.cpu().numpy())

            epoch_loss += loss.item()
            epoch_acc += acc
        
    return epoch_acc / len(data_iterator), epoch_loss / len(data_iterator)

def train_model(model, train_data, valid_data, loss_func, optimizer, epochs=5):

  for epoch in range(epochs):
    
    model.train()
    epoch_loss = 0
    epoch_acc = 0
    
    tic = time.time()
    for inp, label in train_data:
      predictions = model(inp.t()).squeeze(1)
      loss = loss_func(predictions, label)
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      epoch_loss += loss.item()

      break
      
    toc = time.time()
    
#     train_acc, _ = evaluate(model, train_data, loss_func)
#     acc, _ = evaluate(model, valid_data, loss_func)
#     toe = time.time()
#     print('Loss at epoch %d : %f, train acc : %f, valid acc : %f | train time : %d sec, eval time : %d sec' % (epoch, epoch_loss / len(train_data), train_acc, acc, toc-tic, toe - toc))

In [0]:
import torch.nn.functional as F
import torch.optim as optim


SEQ_LEN = 20
INPUT_DIM = 50
OUTPUT_DIM = INPUT_DIM

EMBEDDING_DIM = 64
HIDDEN_DIM = 256

N_LAYERS = 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_data_loader, valid_data_loader, test_data_loader = get_data(SEQ_LEN, INPUT_DIM, device=device, data_size=100000)
# model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
# model = RNN_deep(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS)
model = RNN_deep_bidir(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS)
model = model.to(device)

loss_func = F.cross_entropy
optimizer = optim.Adam(model.parameters(), weight_decay=0.00001)

print(f'The model has {count_parameters(model):,} trainable parameters')

train_model(model, train_data_loader, valid_data_loader, loss_func, optimizer, epochs=1)

The model has 193,714 trainable parameters
torch.Size([2, 64, 256])
torch.Size([64, 512])
